In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install flask pyngrok

In [ ]:
'''!pip uninstall -y pyngrok # pyngrok 라이브러리 제거
!rm -f ngrok ngrok-stable-linux-amd64.zip # 현재 디렉토리 파일 제거
!rm -rf /usr/local/bin/ngrok # 시스템 PATH에 있는 ngrok 제거
!rm -rf /root/.ngrok2/ # ngrok 설정 파일 및 캐시 제거
!rm -rf ~/.pyngrok # pyngrok이 관리하는 ngrok 바이너리 폴더도 강제 삭제
!find / -name ngrok -type f -delete 2>/dev/null # 숨겨진 ngrok 파일까지 찾아서 삭제!

!pip install pyngrok
!wget -q -c https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -qq ngrok-stable-linux-amd64.zip

!chmod +x ngrok

!mkdir -p ~/.pyngrok
!mv ngrok ~/.pyngrok/ngrok

print("✅ ngrok 최신 버전 설치 및 경로 설정 완료!")'''

!ngrok authtoken "333MWQEwoM4mSOfiUqQq1HMiaX9_5Q1AqjRJ8VEhSWjUn7a1j"

print("✅ ngrok 인증 토큰 설정 완료!")


import os
from pyngrok import conf
conf.get_default().ngrok_path = os.path.expanduser("~/.pyngrok/ngrok")

print(f"pyngrok이 사용할 ngrok 바이너리 경로 명시적 지정: {conf.get_default().ngrok_path}")
!/usr/bin/env /usr/local/bin/python3 -m pyngrok version
!/usr/bin/env /root/.pyngrok/ngrok version

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
✅ ngrok 인증 토큰 설정 완료!
pyngrok이 사용할 ngrok 바이너리 경로 명시적 지정: /root/.pyngrok/ngrok
/usr/bin/env: ‘/usr/local/bin/python3’: No such file or directory
/usr/bin/env: ‘/root/.pyngrok/ngrok’: No such file or directory


In [ ]:
from flask import Flask, request, jsonify
from pyngrok import ngrok, conf

import pandas as pd
import numpy as np
import torch
from torch.nn.functional import softmax
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import re

app = Flask(__name__)

model = None
tokenizer = None
data = None

emo_names = ['불평/불만', '환영/호의', '감동/감탄', '지긋지긋', '고마움', '슬픔', '화남/분노', '존경', '기대감', '우쭐댐/무시함', '안타까움/실망', '비장함', '의심/불신', '뿌듯함', '편안/쾌적', '신기함/관심', '아껴주는', '부끄러움', '공포/무서움', '절망', '한심함', '역겨움/징그러움', '짜증', '어이없음', '없음', '패배/자기혐오', '귀찮음', '힘듦/지침', '즐거움/신남', '깨달음', '죄책감', '증오/혐오', '흐뭇함(귀여움/예쁨)', '당황/난처', '경악', '부담/안_내킴', '서러움', '재미없음', '불쌍함/연민', '놀람', '행복', '불안/걱정', '기쁨', '안심/신뢰']

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'use device: {device}')

use device: cpu


In [ ]:
def sanitize_emotion_name(name):
    sanitized = re.sub(r'[^\w가-힣]', '_', name)
    sanitized = re.sub(r'_+', '_', sanitized)
    sanitized = sanitized.strip('_')
    if sanitized and sanitized[0].isdigit():
        sanitized = '_' + sanitized
    if not sanitized:
        sanitized = 'unknown_emotion'
    return sanitized

In [ ]:
def load_resource():
  global model, tokenizer, data

  print(f'model loading: {device}')
  try:
    model_path = "/content/drive/MyDrive/EmoModel/result/checkpoint-12500"
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForSequenceClassification.from_pretrained(model_path)
    model.eval()
    print('loading complete')
  except Exception as e:
    print(f'error: {e}')
    model = None

  print(f'data loading')
  try:
    book_path = '/content/drive/MyDrive/EmoModel/all_reviews_emo.csv'
    data = pd.read_csv(book_path)

    emo_col = 'emotions'

    sanitized_emotion_cols_map = {
        label: f'sentiment_{sanitize_emotion_name(label)}' for label in emo_names}

    for original_label, col_name in sanitized_emotion_cols_map.items():
        data[col_name] = 0.0 # 모든 감정 컬럼을 0으로 초기화 (float 타입으로)

        # 감정 문자열 파싱 함수 (예: '감동/감탄( 98.61%)') 정의
    def parse_and_assign_emotions(row):
      emotion_str = row[emo_col]
      if pd.isna(emotion_str):
        return row # NaN이면 원래 row 반환

      emotion_parts = emotion_str.split(', ') # 콤마와 공백으로 감정 부분을 나눔
      for part in emotion_parts:
                # 정규 표현식을 사용하여 "감정명( NNN%)" 패턴을 정확히 파싱!
                # 감정명, 퍼센트 숫자(소수점 허용)를 추출
          match = re.match(r'(.+?)\(\s*(\d+\.?\d*?)%\)', part.strip())
          if match:
              try:
                  original_emotion_name = match.group(1).strip()
                  percentage = float(match.group(2))


                  if original_emotion_name in emo_names:
                            # 정규화된 컬럼 이름을 사용하여 해당 컬럼에 점수 할당
                      col_to_assign = sanitized_emotion_cols_map[original_emotion_name]
                      row[col_to_assign] = percentage / 100.0
              except Exception as e:
                  print(f"경고: 감정 문자열 파싱 오류 '{part}' - {e}")
      return row

        # book_data의 각 행에 대해 감정 파싱 및 할당 로직 적용 (성능을 위해 .copy() 사용)
    if emo_col in data.columns:
            # SettingWithCopyWarning 방지를 위해 .copy() 사용
            temp_book_data = data.copy()
            # apply 함수를 사용해서 각 행별로 감정 파싱 및 점수 할
            data = temp_book_data.apply(parse_and_assign_emotions, axis=1)
            # 원본 'emotion' 컬럼은 이제 필요 없으니 삭제
            data = data.drop(columns=[emo_col])

    print(f"책 데이터 로딩 및 감정 컬럼 전처리 완료! 총 {len(data)}권의 책 데이터 로드.")
    print("전처리 후 책 데이터 미리보기:\n", data.head())

  except Exception as e:
    print(f"책 데이터 로딩 및 전처리 중 에러 발생: {e}")
    data = None

In [ ]:
load_resource()

model loading: cpu
loading complete
data loading
책 데이터 로딩 및 감정 컬럼 전처리 완료! 총 49417권의 책 데이터 로드.
전처리 후 책 데이터 미리보기:
                  title author         id  rating  \
0  비전공자도 이해할 수 있는 챗GPT    박상길  146290174     9.8   
1  비전공자도 이해할 수 있는 챗GPT    박상길  146290174     9.8   
2  비전공자도 이해할 수 있는 챗GPT    박상길  146290174     9.8   
3  비전공자도 이해할 수 있는 챗GPT    박상길  146290174     9.8   
4  비전공자도 이해할 수 있는 챗GPT    박상길  146290174     9.8   

                                                text category  \
0  -출판사로부터 책을 제공받아 주관적으로 작성한 글입니다-다수의 잠재적 독자인 비전공...       IT   
1  시대가 바뀌고 있습니다. 이제 AI의 시대 누구와도 누군가와도 협력해야할 시대가 온...       IT   
2  이제는 챗GPT의 시대이다?23년부터 시작된 AI시대는 아직 끝날 기미가 보이지 않...       IT   
3  [출판사로부터 도서를 제공받아 작성한 주관적인 리뷰입니다.]챗GPT는 2022년 1...       IT   
4  [출판사로부터 도서를 제공받아 작성한 주관적인 리뷰입니다.]챗GPT로 대표되는 생성...       IT   

   sentiment_불평_불만  sentiment_환영_호의  sentiment_감동_감탄  sentiment_지긋지긋  ...  \
0              0.0              0.0           0.9861             0.0  ...   
1              0.0           

In [ ]:
def get_top_emotions(text, tokenizer, model, emo_names, top_n = 3, threshold=0.15):
  input = tokenizer(text, return_tensors = 'pt', truncation = True, padding = True)
  if torch.cuda.is_available():
    input = {k: v.to("cuda") for k, v in input.items()}

  model.to(device)

  with torch.no_grad():
    output = model(**input)

  logits = output.logits.cpu().numpy()[0]

  probabilities = 1/(1+np.exp(-logits))

  filtered_emo = []
  for i, prob in enumerate(probabilities):
    if prob >= threshold:
      filtered_emo.append((emo_names[i], prob))

  filtered_emo.sort(key=lambda x: x[1], reverse = True)

  top_emo_percent = [
      (name, f"{prob*100: .2f}%")
      for name, prob in filtered_emo[:top_n]
  ]

  return top_emo_percent

In [ ]:
@torch.no_grad()
def analyze_sentiment(text: str) -> dict:
  if model is None:
    return{"error": "not load"}

  try:
    result = get_top_emotions(
        text = text,
        tokenizer = tokenizer,
        model = model,
        emo_names = emo_names,
        top_n = 3,
        threshold = 0.15
    )

    top_n_emotions = {name: percent_str.strip() for name, percent_str in result}

    if not top_n_emotions:
      return {"error": "nothing"}

    return top_n_emotions

  except Exception as e:
    print(f"감정 분석 중 오류 발생: {e}")
    return {"error": f"분석 실패: {e}"}

In [ ]:
def recommend_book(user_sentiment_breakdown: dict, num_recommendations: int = 7) -> list:
  if data is None:
    return []

  user_sentiment_vector = np.array([
      float(user_sentiment_breakdown.get(label, '0%').strip('%'))/100
          if label in user_sentiment_breakdown else 0.0

          for label in emo_names
  ])

  book_sentiment_columns = [f'sentiment_{sanitize_emotion_name(label)}' for label in emo_names]

  missing_cols = [col for col in book_sentiment_columns if col not in data.columns]
  if missing_cols:
     print(f"경고: 책 데이터에 다음 감정 컬럼이 없습니다: {missing_cols}. 추천에 오류가 발생할 수 있습니다.")
     return []

  book_sentiment_vectors = data[book_sentiment_columns].values

  from sklearn.metrics.pairwise import cosine_similarity

  similarities = cosine_similarity(user_sentiment_vector.reshape(1, -1), book_sentiment_vectors)[0]

  sorted_book_indices = np.argsort(similarities)[::-1]

  recommend_list = []

  unique_book = set()

  for i in sorted_book_indices:
    if len(recommend_list) >= num_recommendations:
      break

    book_info = data.iloc[i]

    current_book_title = book_info['title'] if 'title' in book_info.index else '제목 없음'
    current_book_author = book_info['author'] if 'author' in book_info.index else '작가 미상'

    book_identifier = f"{current_book_title}-{current_book_author}" # 고유 식별자 생성

    if book_identifier in unique_book:
        continue # 이미 추천된 책이라면 건너뛰기!

        # 중복되지 않는 책이라면 추가!
    unique_book.add(book_identifier)

    book_emotions = {}
    for label in emo_names:
      sanitized_col_name = sanitize_emotion_name(label)

      if sanitized_col_name in book_info.index and not pd.isna(book_info[sanitized_col_name]):
        book_emotions[label] = f"{book_info[sanitized_col_name]*100: .0f}%"
      else:
        book_emotions[label] = "0%"

    recommend_list.append({
      "title": current_book_title,
      "author": current_book_author,
      "sentiment_breakdown": book_emotions,
      })

  return recommend_list

In [ ]:
@app.route('/recommend', methods=['POST'])
def recommend_books_api():
    print("--- 새로운 책 추천 요청 수신 ---")
    if not request.json or 'user_diary' not in request.json:
        print("경고: 데이터가 요청에 포함되어 있지 않습니다.")
        return jsonify({"error": "No user text provided"}), 400

    text = request.json['user_diary']
    print(f"수신된 사용자 : '{text[:50]}...'") # 긴 일기면 앞부분만 출력

    user_sentiment_breakdown = analyze_sentiment(text)
    if "error" in user_sentiment_breakdown:
        print(f"사용자 감정 분석 실패: {user_sentiment_breakdown['error']}")
        return jsonify(user_sentiment_breakdown), 500

    print(f"사용자 감정 분석 결과: {user_sentiment_breakdown}")

    # ⭐️ 6-2. 분석된 감정을 기반으로 책 추천
    recommended_books = recommend_book(user_sentiment_breakdown, num_recommendations=7) # 7권 추천!

    print(f"총 {len(recommended_books)}권의 책 추천 완료.")

    # ⭐️ 6-3. 추천 결과를 JSON 형태로 플러터 앱에 반환
    return jsonify(recommended_books)

In [ ]:
from threading import Thread
import threading
import time

# Flask 서버를 백그라운드 스레드에서 실행
def run_flask():
    app.run(host='0.0.0.0', port=5000, debug=False, use_reloader=False) # ⭐️ port 5000 사용!

# 백그라운드에서 Flask 서버 시작
flask_thread = threading.Thread(target=run_flask)
flask_thread.daemon = True # 메인 스레드 종료 시 함께 종료되도록 설정
flask_thread.start()

# Flask 서버가 시작될 때까지 잠시 대기 (충분히 기다리도록 3초 이상)
time.sleep(3) # ⭐️ 3초 대기!

# ngrok 터널 설정
try:
    print("\nngrok 터널 생성 중...")
    tunnel = ngrok.connect(5000, bind_tls=True) # ⭐️ port 5000 사용!

    public_url = str(tunnel.public_url)

    print(f"\n🚀 Flask 서버가 실행 중입니다!")
    print(f"📱 Flutter 앱에서 사용할 완전한 URL:")
    print(f"   {public_url}/recommend")
    print(f"\n⭐ 이것을 Flutter 코드에 복사하세요:")
    print(f"   final String _backendUrl = '{public_url}/recommend';")

    import requests
    try:
        test_url = f"{public_url}/recommend"
        print(f"\n🔍 테스트 URL: {test_url}")

        test_response = requests.post(test_url,
                                    json={"user_diary": "오늘 친구랑 놀러가서 정말 즐거웠다! 기분이 너무 좋고 행복해!", "test":True}, # ⭐️ 'test':True 같은 더미 추가
                                    timeout=10) # 타임아웃 추가
        if test_response.status_code == 200:
            print(f"✅ 서버 테스트 성공! 응답: {test_response.json()}")
        else:
            print(f"❌ 서버 테스트 실패: {test_response.status_code}")
            print(f"응답 내용: {test_response.text}")
    except Exception as e:
        print(f"⚠️ 서버 테스트 중 오류: {e}")

    print("\n" + "="*60)
    print("🎯 최종 결과:")
    print(f"   Flutter에서 사용할 URL: {public_url}/recommend")
    print("="*60)

    # 서버를 계속 실행 상태로 유지 (Colab 셀이 계속 실행되도록)
    print(f"\n💡 서버가 계속 실행됩니다. 중지하려면 Runtime → Restart runtime을 클릭하세요.")
    try:
        while True:
            time.sleep(60)  # 1분마다 서버 상태 확인 (자원 사용 방지)
            tunnels_status = ngrok.get_tunnels()
            if tunnels_status:
                print(f"🟢 서버 정상 실행 중... ({time.strftime('%H:%M:%S')})")
            else:
                print("🔴 터널이 끊어졌습니다! Colab 런타임을 재시작하세요.")
                break
    except KeyboardInterrupt:
        print("\n서버를 수동으로 중지합니다...")
    finally:
        ngrok.disconnect(public_url)
        ngrok.kill()
        print("ngrok 터널 및 프로세스 종료.")

except Exception as e:
    print(f"❌ ngrok 터널 설정 또는 Flask 앱 실행 중 치명적인 오류 발생: {e}")

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit



ngrok 터널 생성 중...

🚀 Flask 서버가 실행 중입니다!
📱 Flutter 앱에서 사용할 완전한 URL:
   https://617b242cb73c.ngrok-free.app/recommend

⭐ 이것을 Flutter 코드에 복사하세요:
   final String _backendUrl = 'https://617b242cb73c.ngrok-free.app/recommend';

🔍 테스트 URL: https://617b242cb73c.ngrok-free.app/recommend
--- 새로운 책 추천 요청 수신 ---
수신된 사용자 : '오늘 친구랑 놀러가서 정말 즐거웠다! 기분이 너무 좋고 행복해!...'


INFO:werkzeug:127.0.0.1 - - [16/Oct/2025 00:47:09] "POST /recommend HTTP/1.1" 200 -


사용자 감정 분석 결과: {'기쁨': '96.10%', '즐거움/신남': '93.89%', '행복': '93.26%'}
총 7권의 책 추천 완료.
✅ 서버 테스트 성공! 응답: [{'author': '이치조 미사키', 'sentiment_breakdown': {'감동/감탄': '0%', '경악': '0%', '고마움': '0%', '공포/무서움': '0%', '귀찮음': '0%', '기대감': '0%', '기쁨': '0%', '깨달음': '0%', '놀람': '0%', '당황/난처': '0%', '부끄러움': '0%', '부담/안_내킴': '0%', '불쌍함/연민': '0%', '불안/걱정': '0%', '불평/불만': '0%', '비장함': '0%', '뿌듯함': '0%', '서러움': '0%', '슬픔': '0%', '신기함/관심': '0%', '아껴주는': '0%', '안심/신뢰': '0%', '안타까움/실망': '0%', '어이없음': '0%', '없음': '0%', '역겨움/징그러움': '0%', '우쭐댐/무시함': '0%', '의심/불신': '0%', '재미없음': '0%', '절망': '0%', '존경': '0%', '죄책감': '0%', '즐거움/신남': '0%', '증오/혐오': '0%', '지긋지긋': '0%', '짜증': '0%', '패배/자기혐오': '0%', '편안/쾌적': '0%', '한심함': '0%', '행복': '0%', '화남/분노': '0%', '환영/호의': '0%', '흐뭇함(귀여움/예쁨)': '0%', '힘듦/지침': '0%'}, 'title': '오늘 밤, 세계에서 이 사랑이 사라진다 해도'}, {'author': '셸리 리드', 'sentiment_breakdown': {'감동/감탄': '0%', '경악': '0%', '고마움': '0%', '공포/무서움': '0%', '귀찮음': '0%', '기대감': '0%', '기쁨': '0%', '깨달음': '0%', '놀람': '0%', '당황/난처': '0%', '부끄러움'

...

🟢 서버 정상 실행 중... (05:32:09)
🟢 서버 정상 실행 중... (05:33:09)
🟢 서버 정상 실행 중... (05:34:09)
🟢 서버 정상 실행 중... (05:35:09)
🟢 서버 정상 실행 중... (05:36:09)


INFO:werkzeug:127.0.0.1 - - [16/Oct/2025 05:36:51] "GET /recommend HTTP/1.1" 405 -


🟢 서버 정상 실행 중... (05:37:09)
🟢 서버 정상 실행 중... (05:38:09)
🟢 서버 정상 실행 중... (05:39:09)
🟢 서버 정상 실행 중... (05:40:09)
🟢 서버 정상 실행 중... (05:41:09)
🟢 서버 정상 실행 중... (05:42:09)


INFO:werkzeug:127.0.0.1 - - [16/Oct/2025 05:42:37] "GET /recommend HTTP/1.1" 405 -


🟢 서버 정상 실행 중... (05:43:09)
🟢 서버 정상 실행 중... (05:44:09)
🟢 서버 정상 실행 중... (05:45:09)
🟢 서버 정상 실행 중... (05:46:09)
